In [1]:
!pip install -qU "langchain[google-vertexai]"

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-python-client 1.8.0 requires google-api-core<2dev,>=1.13.0, but you have google-api-core 2.24.1 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.


In [10]:
from typing import Optional, List
from pydantic import BaseModel, Field
from enum import Enum

import pandas as pd
from langchain.chat_models import init_chat_model

In [7]:
f = open("./data/linkedin_cv.txt", "r")
linkedin_target_resume = f.read()

In [28]:
llm = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai", temperature = 0.3)

In [29]:
# Pydantic

class SkillType(str, Enum):
    """ SKill type"""
    TECHNICAL = 'technical'
    SOFT_SKILL = 'soft-skill'

class Skill(BaseModel):
    """Skill required for the job"""

    skill: str = Field(description="Required or preferred skill in candidate. This can be technical or soft skill ")
    skill_type: SkillType = Field(description="type of skill in candidate.")
    experience: int = Field(default=None, description="Experience associated the skill.",)
    is_required: bool = Field(default = True, description="Is this skill required or preferred")    

class Skills(BaseModel):
    """Extracted skills from resume"""
    skills: List[Skill]

class Qualifications(BaseModel):
    """Extract qualifications/ education from resume"""
    degree: str = Field(description="Education backgr")
    
# structured_llm = llm.with_structured_output(Skills)
# structured_llm.invoke(linkedin_target_resume)

In [34]:
system_prompt = """
You are a highly skilled AI assistant specializing in analyzing job descriptions and extracting key requirements. Your goal is to meticulously identify and categorize the essential skills, qualifications, experience, and other attributes sought by employers.

**Your Task:**

Given a job description as input, extract and organize the following information into a structured format. Be comprehensive and specific, avoiding vague or generic terms.

**Output Format:**

Present the extracted information in a clear and organized manner, using the following sections:

*   **Required and preferred Skills (Technical & Soft):**  List specific technical skills (e.g., Python, Java, SQL, AWS, project management) and soft skills (e.g., communication, teamwork, leadership, problem-solving).  Prioritize skills explicitly mentioned as required or essential. For each skill, briefly note the context if specified in the job description (e.g., "Python for data analysis," "Communication skills for client interaction"). If levels are specified include those as well (e.g. Intermediate Python).

*   **Required Qualifications/Education:**  Specify the necessary educational background (e.g., Bachelor's degree in Computer Science, Master's degree in Business Administration) and any required certifications or licenses (e.g., PMP, CPA, AWS Certified Solutions Architect). Be exact.

*   **Required Experience:**  Detail the required years of experience and the specific areas of experience sought (e.g., "5+ years of experience in software development," "3+ years of experience in project management within the healthcare industry"). Note the context of the experience.

*   **Responsibilities (List):**  Briefly list the main responsibilities that are explicitly listed in the job description.

*   **Tools/Technologies:** List specific tools or technologies mentioned in the job description. (e.g. Jira, Confluence, Salesforce)

*   **Other Requirements:** Capture any other mandatory requirements not covered above, such as:
    *   Travel requirements (e.g., "Must be willing to travel up to 25%")
    *   Physical requirements (e.g., "Ability to lift 50 lbs")
    *   Security clearances (e.g., "Must be able to obtain a security clearance")
    *   Work authorization requirements (e.g., "Must be authorized to work in the US")

**Instructions:**

1.  **Read the job description carefully and thoroughly.**
2.  **Focus on explicitly stated requirements.**  Infer requirements only when strongly implied and directly supported by the text.
3.  **Be specific and detailed.** Avoid generalizations. For example, instead of "programming skills," specify "Proficiency in Python and Java."
4.  **Prioritize "must-have" requirements.**  Distinguish between essential requirements and desired or "nice-to-have" qualifications if the job description makes that distinction. If there is a distinction, then add a "Desired Skills/Experience" Section.
5.  **Maintain accuracy.**  Do not add information that is not present in the job description.
6.  **Conciseness is important,** but don't sacrifice accuracy or completeness.
"""

In [35]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt
        ),
        ("human", "{text}"),
    ]
)

In [37]:
runnable = prompt | llm

print(runnable.invoke({"text": (
    f"{linkedin_target_resume}"
    "What skills are present in the CV?"
)}))

content="Here's a breakdown of the requirements extracted from the job description:\n\n*   **Required Skills (Technical & Soft):**\n    *   Java (7+ years of experience, professional and academic)\n    *   Python (7+ years of experience, professional and academic)\n    *   C++ (7+ years of experience, professional and academic)\n    *   Software and networking platforms engineering (5+ years of experience)\n    *   Scripting\n    *   Test automation\n    *   Documentation (design processes, including development, testing, analytics, and troubleshooting)\n    *   Problem-solving (finding technical issues and fixing them)\n    *   Communication (collaboration with a global team)\n    *   Planning\n    *   Analytical skills (analyze the efficiency and feasibility of project operations)\n\n*   **Required Qualifications/Education:**\n    *   None explicitly stated, but a desire to continue professional growth through training and education is mentioned.\n\n*   **Required Experience:**\n    